In [78]:
import os
import pandas as pd
import numpy as np

def trim_first_last_n_seconds(df, n, freq):
    remove_indexes = list(range(0, int(n*1000/freq)))
    df = df.drop(remove_indexes)

    remove_indexes = list(range(df.shape[0] - int(n*1000/freq), df.shape[0]-1))
    df = df.drop(remove_indexes)
    
    return df
    

in_dir = "final"
dfs_list = []

features = []
labels = []

for root, dirs, files in os.walk(in_dir):
    path = root.split(os.sep)

    for f in files:
        print("/".join(path) + "/" + f)

        full_path = "/".join(path) + "/" + f
        df = pd.read_csv(full_path, header=None)
        
        print("Before trimming: ", df.shape)
        df = trim_first_last_n_seconds(df, 10, 4)
        print("After trimming: ", df.shape)
   
        #Sample the data according to the size of the window with 50% overlap
        freq = 4
        window_size = 4*1000
        for index in range(0, df.shape[0]-1000, 500):
            indexes = list(range(index, index + int(window_size/freq)))
            
            window = df.iloc[indexes,:]
            
            #Generate the features for this window
            mean_x = np.mean(window[1].tolist())
            mean_y = np.mean(window[2].tolist())
            mean_z = np.mean(window[3].tolist())
            
            var_x = np.var(window[1].tolist())
            var_y = np.var(window[2].tolist())
            var_z = np.var(window[3].tolist())
        
            #Store the features
            window_feature = []
            window_feature.append(mean_x)
            window_feature.append(mean_y)
            window_feature.append(mean_z)
            window_feature.append(var_x)
            window_feature.append(var_y)
            window_feature.append(var_z)
            
            features.append(window_feature)
            
            #Store the label
            labels.append(df[5].iloc[1])
        
        dfs_list.append(df)
        
dfs = pd.concat(dfs_list)        

final/0_1_android.sensor.accelerometer.data.csv
Before trimming:  (64964, 6)
After trimming:  (59965, 6)
final/10_1_android.sensor.accelerometer.data.csv
Before trimming:  (64675, 6)
After trimming:  (59676, 6)
final/11_1_android.sensor.accelerometer.data.csv
Before trimming:  (66938, 6)
After trimming:  (61939, 6)
final/12_1_android.sensor.accelerometer.data.csv
Before trimming:  (60980, 6)
After trimming:  (55981, 6)
final/13_1_android.sensor.accelerometer.data.csv
Before trimming:  (62506, 6)
After trimming:  (57507, 6)
final/14_1_android.sensor.accelerometer.data.csv
Before trimming:  (65108, 6)
After trimming:  (60109, 6)
final/15_1_android.sensor.accelerometer.data.csv
Before trimming:  (62588, 6)
After trimming:  (57589, 6)
final/16_1_android.sensor.accelerometer.data.csv
Before trimming:  (82181, 6)
After trimming:  (77182, 6)
final/17_1_android.sensor.accelerometer.data.csv
Before trimming:  (83784, 6)
After trimming:  (78785, 6)
final/18_1_android.sensor.accelerometer.data.cs

In [79]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np
from sklearn.preprocessing import normalize

X = np.array(normalize(features))
y = np.array(labels)

kf = KFold(n_splits=5)
kf.get_n_splits(X)

acc = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = LogisticRegression()
    
    model.fit(X_train, y_train)
    
    predicted_labels = model.predict(X_test)
    
    acc.append(accuracy_score(y_test, predicted_labels))
    
print("Accuracy across all folds: ", np.mean(acc))    

Accuracy across all folds:  0.6258635945808957
